## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [5]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.describe())
print(data.info())
data.head()

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
days_employed_avg = data['days_employed'].mean()
total_income_avg = data['total_income'].mean()
dob_years_avg = data['dob_years'].mean()

data.loc[data['children'] == 20,'children'] = 2
data.loc[data['children'] == -1,'children'] = 0


data['dob_years'] = data['dob_years'].fillna(value=dob_years_avg)
data['days_employed'] = data['days_employed'].fillna(value=days_employed_avg)
data['total_income'] = data['total_income'].fillna(value=total_income_avg)


Были пропущенны значения в столбцах:
1. Общий трудовой стаж в днях;
2. Ежемесячный доход;

Это может быть связано с тем, что клиент не своевременно подал Трудовую книжку и справку 2-НДФЛ, либо эти данные не были внесены/утеряны.

Были заполненные все пропущенных значения в двух столбцах, "0" - использовался как неизвестное значение.

<<Первая правка>>

Посчитал среднее значение в столбцах
1. Общий трудовой стаж в днях;
2. Ежемесячный доход;

Подставил эти значения в пропуски.
Количество детей изменил вручную.




### Замена типа данных

In [7]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Изменять типы данных не стал, тк в задании прописано изменить "...вещественный тип данных на целочисленный", а это float64 и int64 соответственно. Но в данной таблице нет столбцов со значением float64.


### Обработка дубликатов

In [8]:
#data = data.drop_duplicates().reset_index(drop=True)
#data['education'].value_counts()
data['education'] = data['education'].str.lower()
data['education'].value_counts()
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Вывод

Сначала было найдено общее количество дубликатов по форме #print(data.duplicated().sum())
Далее были удалены дубликаты методом #drop_duplicates и сохранены индексы методом #reset_index(drop=True)

<<Первая правка>>

С помощью метода value_counts() нашел дубликаты связанные с регистром, и подвел их под единое значение.

__________________________________________________

<<Вторая правка>>

Повторил те же действия что и до приведения столбца с образованием в один регистр

### Лемматизация

In [10]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lemma'] = data['purpose'].apply(lambda x: m.lemmatize(x))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [11]:
data.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"


Создал новую колонку, с ключевыми словами из колонки "Цель кредита"

### Категоризация данных

In [12]:
#data['purpose'].value_counts()

def categorizer_purpose(row):
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'жилье' in row:
        return 'жилье'
    if 'свадьба' in row:
        return 'свадьба'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'

data['purpose_group'] = data['purpose'].apply(categorizer_purpose)


def total_income_group(row):
    total_income = row['total_income']
    
    if total_income <= 60000:
        return 'низкий'
    elif total_income >= 60001 and total_income <=100000:
        return 'средний'
    else:
        return 'высокий'

data['total_income_group'] = data.apply(total_income_group, axis=1)

### Вывод

<font color=blue><<Первая правка>>
 
Стал писать функцию, чтобы группировать уровень доходов, но почему то стал ругаться на purpose_lemma, не могу разобраться почему
_______________________________________
   
    
<font color=black><<Вторая правка>>
    
1. исправил указанные ошибки, появилась новая ('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().', 'occurred at index 0')

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [13]:
def check_debt(row):
   
    debt_ind = row['debt']
   
    if debt_ind == 0:
        return 'no_debt'
    else:
        return 'debt'
 
 
data['debt_info'] = data.apply(check_debt, axis =1)
data_pivot = data.pivot_table(index=['children'], columns = 'debt_info', values = 'debt', aggfunc='count')  
data.loc[data['children'] == 20,'children'] = 2
data.loc[data['children'] == -1,'children'] = 0
 
data_pivot['sum'] = data_pivot.sum(axis=1) 
data_pivot['return_debt'] = (data_pivot['no_debt']/data_pivot['sum'])*100
data_pivot

debt_info,debt,no_debt,sum,return_debt
children,,,,
0,1064.0,13074.0,14138.0,92.474183
1,444.0,4364.0,4808.0,90.765391
2,202.0,1926.0,2128.0,90.507519
3,27.0,303.0,330.0,91.818182
4,4.0,37.0,41.0,90.243902
5,NaN,9.0,9.0,100.000000


### Вывод

Если не брать в расчёт аномалию, то чем меньше детей, тем реже задолженность по возвратам кредитов в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
data['debt_info'] = data.apply(check_debt, axis =1)
 
data_pivot = data.pivot_table(index=['family_status'], columns = 'debt_info', values = 'debt', aggfunc='count')  
 
data_pivot['sum'] = data_pivot.sum(axis=1) 
data_pivot['return_debt'] = (data_pivot['no_debt']/data_pivot['sum'])*100
data_pivot

debt_info,debt,no_debt,sum,return_debt
family_status,,,,
Не женат / не замужем,274,2536,2810,90.249110
в разводе,85,1110,1195,92.887029
вдовец / вдова,63,896,959,93.430657
гражданский брак,388,3763,4151,90.652855
женат / замужем,931,11408,12339,92.454818


### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
data['debt_info'] = data.apply(check_debt, axis =1)
 
data_pivot = data.pivot_table(index=['total_income_group'], columns = 'debt_info', values = 'debt', aggfunc='count')  
 
data_pivot['sum'] = data_pivot.sum(axis=1) 
data_pivot['return_debt'] = (data_pivot['no_debt']/data_pivot['sum'])*100
data_pivot

debt_info,debt,no_debt,sum,return_debt
total_income_group,,,,
высокий,1387,15604,16991,91.836855
низкий,49,757,806,93.920596
средний,305,3352,3657,91.659830


Люди с низким заработком возвращают кредиты в срок чаще, чем люди с высоким и средним. 

<font color=blue><<Первая правка>>

Чтобы выполнить данное упражнение мне необходимо категоризировать данные по уровню дохода и провести работу аналогичной выше.
    Если я ошибаюсь, то прошу направить в нужное русло.

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
def check_debt(row):
   
    debt_ind = row['debt']
   
    if debt_ind == 0:
        return 'no_debt'
    else:
        return 'debt'
 

data['debt_info'] = data.apply(check_debt, axis =1)
data_pivot = data.pivot_table(index='purpose_group', columns = 'debt_info', values = 'debt', aggfunc='count')  
 
data_pivot['sum'] = data_pivot.sum(axis=1) 
data_pivot['return_debt'] = (data_pivot['no_debt']/data_pivot['sum'])*100
data_pivot

debt_info,debt,no_debt,sum,return_debt
purpose_group,,,,
автомобиль,90,882,972,90.740741
жилье,94,1204,1298,92.758089
недвижимость,199,2386,2585,92.301741
образование,236,2463,2699,91.256021
свадьба,64,727,791,91.908976


### Вывод

<font color=blue><<Первая правка>>
    
    Уже второй раз python ругается на столбец purpose_lemma, я пытался удалить пропуски через fillna(), но все равно получается данная ошибка. Александр, укажи мне путь)
__________________________________
    
<font color=black><<Вторая правка>>
    
Изменил purpose_lemma на purpose_group

### Шаг 4. Общий вывод

Гипотезы:
1. Более низкий процент возвратов кредитов в срок у людей имеющих детей связан с экономической нагрузкой, которая приходится на детей и их содержание. Поэтому люди не имеющие детей чаще выплачивают кредиты в срок.
2. Зависимости между семейным положением и возвратом кредитов не наблюдаются.Если брать количественное отношение, а не процентное, люди которые женаты/замужем чаще берут кредиты, чем остальное в разы.
3. Люди с низким доходом, более социально ответственнее поэтому возвращают кредиты в срок чаще, хоть и берут реже остальных групп.
4. Кредиты которые берут на жилье и недвижимость возвращают в срок чаще остальных.

Рекомендация:

Целевой группой банка являются женатые/замужние люди с низким доходом и без детей, которые берут кредит на жилье и недвижимость.
